<a href="https://colab.research.google.com/github/BautistaDavid/ColombianFootballLeagueStudies/blob/main/notebooks/dimayor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install great_tables
import pandas as pd
# from great_tables import GT, style_color_gradient
from great_tables import GT


In [3]:
url_fixtures = 'https://raw.githubusercontent.com/BautistaDavid/ColombianFootballLeagueStudies/refs/heads/main/data/scores_fixtures/2025/resultados_dimayor_2025.csv'
url_final_rank = 'https://raw.githubusercontent.com/BautistaDavid/ColombianFootballLeagueStudies/refs/heads/main/data/final_ranking/2025-2/final_rank_202502.csv'


In [4]:

fixtures = pd.read_csv(url_fixtures)
fixtures.columns = fixtures.columns.str.lower()
fixtures = fixtures[['round','date','time','home','away','score','location']]
fixtures['home_goals'] = fixtures['score'].apply(lambda x: x.split('–')[0]).astype('int')
fixtures['away_goals'] = fixtures['score'].apply(lambda x: x.split('–')[1]).astype('int')
fixtures = fixtures[fixtures['round'].str.contains('Finalización')]

final_rank = pd.read_csv(url_final_rank)

In [5]:
temp_away = (
    fixtures
    .groupby('away')
    .agg(
        away_matches=('away_goals', 'count'),
        away_goals_scored=('away_goals', 'sum'),
        away_goals_conceded=('home_goals', 'sum')
    )
    .reset_index()
)

temp_away['ratio_away_goals_scored'] = temp_away['away_goals_scored'] / temp_away['away_matches']
temp_away['ratio_away_goals_conceded'] = temp_away['away_goals_conceded'] / temp_away['away_matches']

temp_home = (
    fixtures
    .groupby('home')
    .agg(
        home_matches=('home_goals', 'count'),
        home_goals_scored=('home_goals', 'sum'),
        home_goals_conceded=('away_goals', 'sum')
    )
    .reset_index()
)

temp_home['ratio_home_goals_scored'] = temp_home['home_goals_scored'] / temp_home['home_matches']
temp_home['ratio_home_goals_conceded'] = temp_home['home_goals_conceded'] / temp_home['home_matches']


temp_away = temp_away.rename(columns={'away': 'team'})
temp_home = temp_home.rename(columns={'home': 'team'})

temp = temp_home.merge(temp_away,on='team')
temp['total_matches'] = temp['home_matches'] + temp['away_matches']
temp = temp[[
    'team',
    'total_matches',
    'ratio_home_goals_scored',
    'ratio_away_goals_scored',
    'ratio_home_goals_conceded',
    'ratio_away_goals_conceded'
]]

temp.loc[temp['team'] == 'Rionegro','team'] = 'Águilas Doradas'
temp = temp.merge(final_rank,how='left',on='team').sort_values(by='rank_n')
round_cols = temp.drop(columns=
 ['team','total_matches','rank_label','rank_n']).columns

temp[round_cols] = temp[round_cols].round(2)

temp['rank_label'] = temp['rank_label'].str.replace(' Cuadrangular','')
temp['rank_label'] = temp['rank_label'].str.replace('Tabla General','')
temp['rank_label'] = temp['rank_label'].str.replace('Subcampeón','🥈')
temp['rank_label'] = temp['rank_label'].str.replace('Campeón','🥇')



temp.columns = ['Equipo','# Partidos','Ratio Goles \nLocal','Ratio Goles\n Visitante',
                'Ratio Goles \nRecibidos Local','Ratio Goles \nRecibidos Visitante',
                'Ranking','rank_n']
temp = temp[
    ['Ranking','Equipo','# Partidos','Ratio Goles \nLocal','Ratio Goles\n Visitante',
                'Ratio Goles \nRecibidos Local','Ratio Goles \nRecibidos Visitante']
]


In [81]:
temp_ = temp.query("Equipo != 'Afafasdlianza'")
# temp_["icon"] = temp["Equipo"].map({
#     "Alianza": '<img src="https://raw.githubusercontent.com/BautistaDavid/ColombianFootballLeagueStudies/refs/heads/main/data/escudos/Alianza.png" width="20">',
# })

def create_url_image(team):
  team = team.replace(' ','%20') + '.png'
  return f'<img src="https://raw.githubusercontent.com/BautistaDavid/ColombianFootballLeagueStudies/refs/heads/main/data/escudos/{team}" width="25">'

temp_[' '] = temp_['Equipo'].apply(create_url_image)
temp_ = temp_[[' '] + temp_.columns[:-1].tolist()]
temp_ = temp_.drop(columns='# Partidos')
temp_.columns = [' ', 'Ranking', 'Team', '🏠',
       '✈️', '🏠 ', '✈️ ']

In [112]:
g = GT(temp_).tab_header(title="Goals Ratio (Scored/Conceded)", subtitle=f"Liga DIMAYOR - Colombia 2025 - 2").fmt_markdown(
    columns=" ").tab_spanner(label="Scored",
        columns=["🏠", "✈️"]).tab_spanner(label="Conceded",
        columns=["🏠 ", "✈️ "]).data_color(
            palette=["white", "green"], domain=[0, 3],columns=['🏠','✈️']).data_color(
                palette=["white", "red"], domain=[0, 3],columns=['🏠 ','✈️ ']).tab_source_note(
        source_note="* 🏠: Home     ✈️: Away"
    ).tab_source_note(
        source_note="*Ratio: Average goals per match"
    )

# g.save("goal_ratios_table.html")



In [109]:
g

GT(_tbl_data=                                                      Ranking  \
8   <img src="https://raw.githubusercontent.com/Ba...       🥇   
18  <img src="https://raw.githubusercontent.com/Ba...       🥈   
4   <img src="https://raw.githubusercontent.com/Ba...     2 A   
3   <img src="https://raw.githubusercontent.com/Ba...     2 B   
12  <img src="https://raw.githubusercontent.com/Ba...     3 A   
17  <img src="https://raw.githubusercontent.com/Ba...     3 B   
7   <img src="https://raw.githubusercontent.com/Ba...     4 A   
6   <img src="https://raw.githubusercontent.com/Ba...     4 B   
1   <img src="https://raw.githubusercontent.com/Ba...      9    
16  <img src="https://raw.githubusercontent.com/Ba...     10    
13  <img src="https://raw.githubusercontent.com/Ba...     11    
11  <img src="https://raw.githubusercontent.com/Ba...     12    
10  <img src="https://raw.githubusercontent.com/Ba...     13    
0   <img src="https://raw.githubusercontent.com/Ba...     14    
19  <img src="https://raw.githubusercontent.com/Ba...     15    
5   <img src="https://raw.githubusercontent.com/Ba...     16    
14  <img src="https://raw.githubusercontent.com/Ba...     17    
15  <img src="https://raw.githubusercontent.com/Ba...     18    
2   <img src="https://raw.githubusercontent.com/Ba...     19    
9   <img src="https://raw.githubusercontent.com/Ba...     20    

               Team     🏠    ✈️    🏠    ✈️   
8            Junior  2.07  1.29  0.79  1.43  
18           Tolima  1.36  1.07  0.79  0.79  
4        CD América  1.54  0.85  0.92  1.15  
3    CA Bucaramanga  1.62  1.08  0.69  1.00  
12         Nacional  1.92  1.54  1.00  1.38  
17         Santa Fe  1.23  0.85  0.69  1.00  
7     Independiente  2.00  1.92  1.15  1.54  
6      Fortaleza FC  1.15  1.00  0.77  1.46  
1           Alianza  1.30  1.10  0.60  1.60  
16  Águilas Doradas  1.70  1.20  1.30  1.40  
13      Once Caldas  1.40  0.70  1.40  1.00  
11      Millonarios  1.40  0.90  1.00  1.60  
10         Llaneros  0.70  1.00  1.00  1.50  
0           AD Cali  0.90  1.30  1.50  1.30  
19  Unión Magdalena  1.50  0.70  1.40  1.80  
5          Envigado  0.60  1.20  1.00  1.20  
14            Pasto  1.90  0.70  1.30  1.50  
15          Pereira  1.50  0.40  1.90  1.70  
2      Boyacá Chicó  0.80  0.60  0.80  2.10  
9        La Equidad  0.70  0.70  1.00  1.90  , _body=<great_tables._gt_data.Body object at 0x78f3c2add670>, _boxhead=Boxhead([ColInfo(var=' ', type=<ColInfoTypeEnum.default: 1>, column_label=' ', column_align='left', column_width=None), ColInfo(var='Ranking', type=<ColInfoTypeEnum.default: 1>, column_label='Ranking', column_align='left', column_width=None), ColInfo(var='Team', type=<ColInfoTypeEnum.default: 1>, column_label='Team', column_align='left', column_width=None), ColInfo(var='🏠', type=<ColInfoTypeEnum.default: 1>, column_label='🏠', column_align='right', column_width=None), ColInfo(var='✈️', type=<ColInfoTypeEnum.default: 1>, column_label='✈️', column_align='right', column_width=None), ColInfo(var='🏠 ', type=<ColInfoTypeEnum.default: 1>, column_label='🏠 ', column_align='right', column_width=None), ColInfo(var='✈️ ', type=<ColInfoTypeEnum.default: 1>, column_label='✈️ ', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x78f3c2a94350>, _spanners=Spanners([SpannerInfo(spanner_id='Scored', spanner_level=0, spanner_label='Scored', spanner_units=None, spanner_pattern=None, vars=['🏠', '✈️'], built=None), SpannerInfo(spanner_id='Conceded', spanner_level=0, spanner_label='Conceded', spanner_units=None, spanner_pattern=None, vars=['🏠 ', '✈️ '], built=None)]), _heading=Heading(title='Goals Ratio (Scored/Conceded)', subtitle='Liga DIMAYOR - Colombia 2025 - 2', preheader=None), _stubhead=None, _summary_rows=<great_tables._gt_data.SummaryRows object at 0x78f3c2adcf20>, _summary_rows_grand=<great_tables._gt_data.SummaryRows object at 0x78f3c2add010>, _source_notes=['* 🏠: Home     ✈️: Away', '*Ratio: Average goals per match'], _footnotes=[],